---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    average_clustering = [nx.average_clustering(G) for G in P1_Graphs]
    average_shortest_path_length = [nx.average_shortest_path_length(G) for G in P1_Graphs]
    
    graph_ident_list = []
    for i, G in enumerate(P1_Graphs):
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        if len(histogram) > 10:
            graph_ident_list.append('PA')
        elif average_clustering[i] <= 0.1:
            graph_ident_list.append('SW_H')
        else:
            graph_ident_list.append("SW_L")
            
    return graph_ident_list # Your Answer Here

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['BetwCent']   = pd.Series(nx.betweenness_centrality(G))
    df['ClosenessCent'] = pd.Series([v for i,v in nx.closeness_centrality(G).items()])
    df['ClustCoeff'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(nx.degree(G))
    df['DegreeCent'] = pd.Series(nx.degree_centrality(G))
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))

    df_X_pred = df.iloc[:,:-1][df['ManagementSalary'].isnull()]
    df_X_train = df.iloc[:,:-1][~df['ManagementSalary'].isnull()]
    df_Y_train = df['ManagementSalary'][~df['ManagementSalary'].isnull()]
    
    gb_clf = GradientBoostingClassifier(n_estimators=500, random_state=0)
    gb_clf.fit(df_X_train, df_Y_train)
    y_proba = gb_clf.predict_proba(df_X_pred)[:,1]
    
    return pd.Series(y_proba, index=df_X_pred.index.values) # Your Answer Here

In [9]:
salary_predictions()

1       0.000013
2       0.999911
5       0.999929
8       0.007093
14      0.001216
18      0.000955
27      0.001424
30      0.992288
31      0.091689
34      0.000788
37      0.001719
40      0.000548
45      0.000032
54      0.068609
55      0.963997
60      0.008964
62      0.999923
65      0.999809
77      0.000101
79      0.000148
97      0.000133
101     0.000021
103     0.413314
108     0.000444
113     0.012114
122     0.000010
141     0.063556
142     0.999923
144     0.000838
145     0.887801
          ...   
913     0.000973
914     0.000163
915     0.000006
918     0.000875
923     0.001372
926     0.013228
931     0.000033
934     0.000002
939     0.000006
944     0.000003
945     0.001372
947     0.004599
950     0.000737
951     0.000089
953     0.000073
959     0.000004
962     0.000010
963     0.000027
968     0.001175
969     0.001593
974     0.000746
984     0.000003
987     0.000572
989     0.003348
991     0.001500
992     0.000004
994     0.000004
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

def new_connections_predictions():
    
    
    # Your Code Here
    Preferential_Attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in Preferential_Attachment])
    df['PrefAttachment'] = [x[2] for x in Preferential_Attachment]
    df['CNSoundHop'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G, community='Department'))]
    df['RAI'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['JaccardCoeff'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df['AdamicAdarIndex'] = [x[2] for x in list(nx.adamic_adar_index(G))]
    df['CommonNeighbors'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in df.index.values]
    df['RASoundHop'] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))]
    df = df.merge(future_connections, left_index=True, right_index=True)

    df_X_pred = df.iloc[:,:-1][df['Future Connection'].isnull()]
    df_X_train = df.iloc[:,:-1][~df['Future Connection'].isnull()]
    df_Y_train = df['Future Connection'][~df['Future Connection'].isnull()]    
    
    gb_clf = GradientBoostingClassifier(n_estimators=200, random_state=0)
    gb_clf.fit(df_X_train, df_Y_train)
    y_proba = gb_clf.predict_proba(df_X_pred)[:,1]
    
    return pd.Series(y_proba, index=df_X_pred.index.values) # Your Answer Here

In [12]:
new_connections_predictions()

(0, 9)          0.029014
(0, 19)         0.061971
(0, 20)         0.230222
(0, 35)         0.018903
(0, 38)         0.016220
(0, 42)         0.865847
(0, 43)         0.014841
(0, 50)         0.013557
(0, 53)         0.539053
(0, 54)         0.039381
(0, 62)         0.292737
(0, 63)         0.052714
(0, 69)         0.027677
(0, 72)         0.012736
(0, 83)         0.176082
(0, 90)         0.025968
(0, 91)         0.018174
(0, 95)         0.108120
(0, 100)        0.029014
(0, 106)        0.988205
(0, 108)        0.018890
(0, 109)        0.012736
(0, 110)        0.012736
(0, 118)        0.015604
(0, 122)        0.025968
(0, 131)        0.038544
(0, 133)        0.781016
(0, 140)        0.064339
(0, 149)        0.131044
(0, 151)        0.016220
                  ...   
(988, 989)      0.012736
(988, 996)      0.012736
(988, 997)      0.049644
(988, 1000)     0.012736
(988, 1002)     0.012736
(989, 994)      0.012736
(989, 996)      0.012736
(989, 1003)     0.012736
(989, 1004)     0.012736
